In [1]:
import sqlite3
import pandas as pd
import numpy as np
import scipy as sc
import yaml
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from math import isnan

%matplotlib inline

#import warnings
#warnings.filterwarnings("ignore")

In [2]:
# Lendo as configurações
with open('.\configurations.yml') as file:
    configurations = yaml.load(file, Loader=yaml.FullLoader)

In [3]:
db = sqlite3.connect(configurations["Database"]["Path"]) # connecting to sqlite database

In [4]:
df = pd.read_sql_query("SELECT * FROM CARTOES", db)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62685 entries, 0 to 62684
Data columns (total 59 columns):
Unnamed: 0                                 62685 non-null int64
index                                      62685 non-null int64
id                                         62685 non-null int64
mesExtrato                                 62685 non-null object
dataTransacao                              62685 non-null object
valorTransacao                             62685 non-null float64
id_tipoCartao                              62685 non-null int64
codigo_tipoCartao                          62685 non-null int64
descricao_tipoCartao                       62685 non-null object
numeroInscricaoSocial                      10635 non-null float64
nome_estabelecimento                       62685 non-null object
razaoSocialReceita                         60884 non-null object
nomeFantasiaReceita                        38874 non-null object
localidadePessoa                           62685 

In [18]:
#criação de marcadores para contagem de repetição de compras em um mesmo CPF ou CNPJ, por usuário

df['marcador_cnpj']=np.nan
df['marcador_cpf']=np.nan

In [19]:
df['tamanho_codigoFormatado_estabelecimento'] = df.apply(lambda x : len(x['codigoFormatado_estabelecimento']),axis=1)#se for 14 é cpf e se for 18 é cnpj

In [22]:
%time
for i in range(0,62685,1):#contagem de linha do df
    portador=df['codigoFormatado_portador'][i] #portador a ser procurado
    estabelecimento=df['codigoFormatado_estabelecimento'][i]#estabelecimento a ser procurado
    if isnan(df['marcador_cnpj'][i])==True & isnan(df['marcador_cpf'][i])==True:#ainda não contou esse usuário e empresa, não tem marcador de cpf nem cnpj nessa linha
        if df['tamanho_codigoFormatado_estabelecimento'][i]==18:#o vendedor tem CNPJ
            marcador=0
            for j in range(i,62685,1):
                if(df['codigoFormatado_portador'][j]==portador):#se encontrar um gasto do portador
                    if(df['codigoFormatado_estabelecimento'][j]==estabelecimento):
                        df['marcador_cnpj'][j]=marcador
                        j=j+1
                        marcador=marcador+1
        if df['tamanho_codigoFormatado_estabelecimento'][i]==14:#o vendedor tem CPF, pessoa física
            marcador=0
            for j in range(i,62685,1):
                if(df['codigoFormatado_portador'][j]==portador):#se encontrar um gasto do portador
                    if(df['codigoFormatado_estabelecimento'][j]==estabelecimento):
                        df['marcador_cpf'][j]=marcador
                        j=j+1
                        marcador=marcador+1 

Wall time: 0 ns


C:\Users\debcz\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\debcz\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [25]:
df['marcador_cnpj'].value_counts()

0.0       16835
1.0        3662
2.0        1372
3.0         670
4.0         377
          ...  
8440.0        1
8389.0        1
8354.0        1
8275.0        1
8330.0        1
Name: marcador_cnpj, Length: 8549, dtype: int64

In [27]:
df['marcador_cpf'].value_counts()

0.0    1471
1.0     263
2.0      39
3.0      16
5.0       5
4.0       5
7.0       1
6.0       1
Name: marcador_cpf, dtype: int64

In [26]:
df.to_csv(r'C:\Users\debcz\Documents\GitHub\Public-Spending-Analisys\src\database_counts.csv')

In [ ]:
df=pd.read_csv(r'C:\Users\debcz\Documents\GitHub\Public-Spending-Analisys\src\database_counts.csv')

In [7]:
df["valorTransacao"] 

0           9.00
1          20.00
2         609.38
3         389.60
4          14.99
          ...   
62680     544.40
62681     339.48
62682     306.25
62683    1145.17
62684     640.88
Name: valorTransacao, Length: 62685, dtype: float64

In [8]:
df['gastos_mesmo_valor']=np.nan 

In [14]:
#marcador de gastos de mesmo valor de um mesmo portador
%time
for i in range(0,62685,1):#contagem de linha do df
    portador=df['codigoFormatado_portador'][i] #portador a ser procurado
    valor=df["valorTransacao"][i]#valor 
    if isnan(df['gastos_mesmo_valor'][i])==True :#ainda não contabilizou esse valor desse portador
        marcador=0
        for j in range(i,62685,1):
            if(df['codigoFormatado_portador'][j]==portador):#se encontrar um gasto do portador
                if(df["valorTransacao"][j]==valor):
                    df['gastos_mesmo_valor'][j]=marcador
                    j=j+1
                    marcador=marcador+1
       

Wall time: 0 ns


C:\Users\debcz\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


KeyboardInterrupt: 

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
Name: gastos_mesmo_valor, dtype: float64